## Временные ряды 2. Преобразование признаков 

p - глубина истории, сколько измерений назад учитываем в моделях   
h - горизонт прогноза , на сколько измерений вперед смотрим при моделировании (как далеко предсказываем)

__1. Прочитайте базу "Потребление Энергии" и сделайте модель для сезонных изменений по data_exp_1 с глубиной прогноза h=1 и h=10 по образцу из примера для обработки ряда "цена Нефти" .__



In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')



In [2]:
data = pd.read_csv('../lection2/my_PJME_MW.csv', index_col=[0], parse_dates=[0])


In [3]:
def exponential_smoothing(series, alpha):
    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result

d_data=data.resample('D').mean()
data_exp_1 = exponential_smoothing(d_data['PJME_MW']  , 0.05)

In [4]:
d_ar = d_data['PJME_MW'].values
d_s = pd.DataFrame(data = d_data[['PJME_MW']])


In [5]:
data_exp_1 = pd.DataFrame(data = data_exp_1, index = d_s.index)

In [6]:
df = pd.DataFrame(data_exp_1)
df['t'] = 0
df.iloc[1:,1]=df.iloc[:-1,0].values
df.fillna(0)
df['exp_0_01'] = exponential_smoothing(df[0], 0.01)
df['exp_0_03'] = exponential_smoothing(df[0], 0.03)
df['exp_0_08'] = exponential_smoothing(df[0], 0.08)
df['diff1'] = 0
df['diff1'].iloc[1:] = df.iloc[:-1,0]-df.iloc[1:,0]

df.head()

,0,t,exp_0_01,exp_0_03,exp_0_08,diff1
Date,,,,,,
2002-01-01,31080.739130,0.000000,31080.739130,31080.739130,31080.739130,0.0
2002-01-02,31239.779257,31080.739130,31082.329532,31085.510334,31093.462341,0.0
2002-01-03,31403.384044,31239.779257,31085.540077,31095.046546,31118.256077,0.0
2002-01-04,31518.987759,31403.384044,31089.874554,31107.764782,31150.314611,0.0
2002-01-05,31463.294621,31518.987759,31093.608754,31118.430677,31175.353012,0.0


In [7]:
def split_data_d( data, split_date ):
    return data.loc[data.index.get_level_values('Date') <= split_date].copy(), \
           data.loc[data.index.get_level_values('Date') >  split_date].copy()

train_d, test_d = split_data_d( df, '10-10-2017')


In [8]:
model_gb1 = GradientBoostingRegressor(max_depth=15, random_state=0, n_estimators=100)
h=1
X_train_d1 = train_d.iloc[:-h,:]
y_train_d1 = train_d[df.columns[0]].values[h:]
X_test_d1 = test_d.iloc[:-h,:]
y_test_d1 = test_d[df.columns[0]].values[h:]

model_gb1.fit( X_train_d1, y_train_d1) 
X_test_pred_gb1 = model_gb1.predict(X_test_d1)


In [9]:
model_gb10 = GradientBoostingRegressor(max_depth=15, random_state=0, n_estimators=100)
h=10
X_train_d10 = train_d.iloc[:-h,:]
y_train_d10 = train_d[df.columns[0]].values[h:]
X_test_d10 = test_d.iloc[:-h,:]
y_test_d10 = test_d[df.columns[0]].values[h:]

model_gb10.fit( X_train_d, y_train_d ) 
X_test_pred_gb10 = model_gb10.predict(X_test_d)


NameError: name 'X_train_d' is not defined

__2. Оцените ошибку модели из п.2 .__


In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

er_g1 = mean_absolute_percentage_error(y_true=y_test_d1, y_pred=X_test_pred_gb1)
er_g10 = mean_absolute_percentage_error(y_true=y_test_d10, y_pred=X_test_pred_gb10)
print('ошибка градиентного бустинга при h 1:  ', er_g1, '%')
print('ошибка градиентного бустинга при h 10:  ', er_g10, '%')


__3. Сделайте вывод о том, как изменилась ошибка модели по значениям h.__

При h=10 модель стала хуже

__4. Попробуйте предположить характер ряда (стационарный или нет, с трендом, с сезоном, с циклом) для рядов из списка и аргументируйте свой ответ (одна строка):   
    а) Изменение температуры за окном :).   
    б) Количество денег, которые тратят покупатели в интернет магазине.   
    в) Вес растущего котенка.   
    г) Прирост объема деревьев в лесу от времени__
    
    
Ответ:    
    a) Не является, т.к. присутвует сезонность   
    б) Не является. Сезонность, в виде праздников, выходных и т.д.
    в) Не является. Присутвует тренд.
    д) Можно сказать что является. Т.к. на прирост деревьев в лесу влияет множество внешних процессов. 


    


